In [1]:
import os
import pandas as pd
import librosa
import traceback

def analyze_audio(file_path):
    """Analiza un archivo de audio y retorna el BPM y la clave."""
    try:
        # Cargar el archivo de audio
        y, sr = librosa.load(file_path, sr=None)
        # Detectar BPM (tempo)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr, hop_length=512)
        # Detectar clave (Key)
        key = librosa.feature.chroma_cqt(y=y, sr=sr).mean(axis=1).argmax()
        return tempo, librosa.midi_to_note(key)
    except Exception as e:
        print(f"Error al analizar {file_path}: {e}")
        traceback.print_exc()  # Muestra el detalle completo del error
        return None, None

def save_library(library_df_dict, output_file):
    """Guarda el archivo Excel actualizado."""
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        for sheet_name, df in library_df_dict.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"Archivo Excel guardado en {output_file}")

def update_biblioteca_with_missing_values(library_file, output_file, max_updates=100, save_interval=10):
    # Leer todas las hojas del archivo "Biblioteca.xlsx"
    library_df_dict = pd.read_excel(library_file, sheet_name=None)

    updated_song_details = []

    # Contador de actualizaciones
    updates_count = 0

    # Recorrer todas las hojas de "Biblioteca.xlsx"
    for sheet_name, library_df in library_df_dict.items():
        print(f"Procesando hoja: {sheet_name}")

        if 'Titulo' in library_df.columns and 'Ubicacion' in library_df.columns and 'Nombre' in library_df.columns:
            library_df['Titulo'] = library_df['Titulo'].apply(lambda x: x.replace("’", "'").replace(".mp3", ""))
            library_df['Nombre'] = library_df['Nombre'].apply(lambda x: x.replace("’", "'").replace(".mp3", ""))

            # Buscar títulos sin BPM o Clave / Key
            for index, row in library_df.iterrows():
                if (pd.isna(row['BPM']) or pd.isna(row['Clave / Key'])) and updates_count < max_updates:
                    # Verificar ambas posibles rutas de archivo
                    file_path = os.path.join(row['Ubicacion'], row['Nombre'] + ".mp3")
                    if not os.path.exists(file_path):
                        file_path = os.path.join(row['Ubicacion'], row['Titulo'] + ".mp3")
                    
                    if os.path.exists(file_path):
                        bpm, key = analyze_audio(file_path)
                        if pd.isna(row['BPM']) and bpm:
                            library_df.at[index, 'BPM'] = bpm
                        if pd.isna(row['Clave / Key']) and key:
                            library_df.at[index, 'Clave / Key'] = key
                        updated_song_details.append({
                            'Nombre': row['Nombre'],
                            'Titulo': row['Titulo'],
                            'Ubicacion': row['Ubicacion'],
                            'BPM': bpm if pd.isna(row['BPM']) else row['BPM'],
                            'Clave / Key': key if pd.isna(row['Clave / Key']) else row['Clave / Key']
                        })
                        updates_count += 1
                        print(f"Actualizado: Nombre={row['Nombre']}, Titulo={row['Titulo']}, BPM={bpm}, Clave / Key={key}")
                        
                        # Guardar cada 10 actualizaciones
                        if updates_count % save_interval == 0:
                            save_library(library_df_dict, output_file)
                    else:
                        print(f"Archivo no encontrado: {file_path}")

                    if updates_count >= max_updates:
                        break  # Salir del bucle después de actualizar el número máximo de canciones

            # Actualizar el DataFrame en el diccionario
            library_df_dict[sheet_name] = library_df
        else:
            print(f"La hoja '{sheet_name}' no tiene las columnas necesarias 'Titulo', 'Nombre' y 'Ubicacion'.")

        if updates_count >= max_updates:
            break  # Salir del bucle después de actualizar el número máximo de canciones

    # Guardar el archivo actualizado final
    save_library(library_df_dict, output_file)

    print(f"Archivo Excel actualizado con BPM y Clave / Key para {updates_count} canciones.")
    return updated_song_details

# Configuraciones
library_file = r"E:\Musica-E\Organizacion\Biblioteca.xlsx"
output_file = library_file  # Sobrescribir el archivo original "Biblioteca.xlsx"
max_updates = 2000  # Número máximo de canciones a actualizar
save_interval = 10  # Guardar el archivo cada 10 actualizaciones

# Actualizar el archivo con BPM y Clave / Key generados para hasta 100 canciones sin estos valores
updated_song_details = update_biblioteca_with_missing_values(library_file, output_file, max_updates, save_interval)

# Imprimir los detalles de las canciones actualizadas
if updated_song_details:
    print("Detalles de las canciones actualizadas:")
    for song in updated_song_details:
        print(f"Nombre: {song['Nombre']}, Título: {song['Titulo']}, Ubicación: {song['Ubicacion']}, BPM: {song['BPM']}, Clave / Key: {song['Clave / Key']}")
else:
    print("No se encontraron canciones sin BPM o Clave / Key para actualizar.")


Procesando hoja: Sheet1
Error al analizar E:\Musica-E\DJ Project\Multi Culti Party\mix\Mix_VicoDJ_-_Cuentale.mp3: 


Traceback (most recent call last):
  File "C:\Users\aleja\AppData\Local\Temp\ipykernel_35160\2100267759.py", line 12, in analyze_audio
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr, hop_length=512)
  File "C:\Users\aleja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\librosa\beat.py", line 230, in beat_track
    bpm = _tempo(
  File "C:\Users\aleja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\librosa\feature\rhythm.py", line 415, in tempo
    tg = tempogram(
  File "C:\Users\aleja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\librosa\feature\rhythm.py", line 175, in tempogram
    autocorrelate(odf_frame * ac_window, axis=-2), norm=norm, axis=-2
  File "C:\Users\aleja\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\l

Actualizado: Nombre=Mix_VicoDJ_-_Cuentale, Titulo=Mix_VicoDJ_-_Cuentale, BPM=None, Clave / Key=None
Archivo Excel guardado en E:\Musica-E\Organizacion\Biblioteca.xlsx
Archivo Excel actualizado con BPM y Clave / Key para 1 canciones.
Detalles de las canciones actualizadas:
Nombre: Mix_VicoDJ_-_Cuentale, Título: Mix_VicoDJ_-_Cuentale, Ubicación: E:\Musica-E\DJ Project\Multi Culti Party\mix, BPM: None, Clave / Key: None


In [1]:
import librosa
import soundfile

print("Librosa version:", librosa.__version__)
print("Soundfile version:", soundfile.__version__)


Librosa version: 0.10.2.post1
Soundfile version: 0.12.1
